Block 1 — Preparation (imports, seeds, folders, logger)
Here is a single, executable cell that:

imports the necessary libraries,

sets the seeds,

creates the data/, results/, and logs/ folders,

delete /content/sample_data if it exists,

configures a logger with console and file output,

initializes a log file, logs/logs.csv, and appends it to logs/summary.md,

uses timezone-aware timestamps (without the deprecated utcnow()),

avoids escaping warnings with raw strings.

In [1]:
# ============================================================
# ⚙️ Installation des dépendances du projet
# Cette cellule garantit que toutes les librairies nécessaires sont installées.
# ============================================================

import subprocess
import sys

def install_requirements(file_path="requirements.txt"):
    """Installe les paquets listés dans requirements.txt."""
    print(f"Installation/Mise à jour des dépendances via {file_path}...")
    try:
        # Exécute la commande pip
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", file_path])
        print("\n✅ Toutes les dépendances ont été installées ou mises à jour avec succès.")
        print("Veuillez REDÉMARRER le noyau (kernel) du notebook si c'est la première exécution.")
    except subprocess.CalledProcessError as e:
        print(f"\n❌ ERREUR lors de l'installation des dépendances : {e}")

# Exécuter l'installation
install_requirements()


Installation/Mise à jour des dépendances via requirements.txt...

✅ Toutes les dépendances ont été installées ou mises à jour avec succès.
Veuillez REDÉMARRER le noyau (kernel) du notebook si c'est la première exécution.


In [2]:
# Bloc 1 — Préparation
# Imports, seeds, dossiers, logger, journaux init

import os
import csv
import shutil
import random
import logging
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# =========================
# Seeds et conventions
# =========================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# =========================
# Dossiers
# =========================
os.makedirs('data', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('logs', exist_ok=True)

# Supprimer sample_data si présent (environnements Colab)
sample_data_path = '/content/sample_data'
if os.path.isdir(sample_data_path):
    try:
        shutil.rmtree(sample_data_path)
    except Exception as e:
        # Silencieux mais on loguera plus bas
        pass

# =========================
# Timestamps et helpers
# =========================
def utc_timestamp():
    # Timezone-aware ISO 8601
    return datetime.now(timezone.utc).isoformat()

LOG_CSV_PATH = os.path.join('logs', 'logs.csv')
SUMMARY_MD_PATH = os.path.join('logs', 'summary.md')

# Init du fichier logs.csv si vide
if not os.path.isfile(LOG_CSV_PATH):
    with open(LOG_CSV_PATH, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['timestamp', 'level', 'message'])

# =========================
# Logger
# =========================
logger = logging.getLogger('TlogV01')
logger.setLevel(logging.INFO)
logger.handlers.clear()

# Console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_fmt = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
console_handler.setFormatter(console_fmt)
logger.addHandler(console_handler)

# Fichier handler (logs/app.log)
file_handler = logging.FileHandler(os.path.join('logs', 'app.log'))
file_handler.setLevel(logging.INFO)
file_fmt = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
file_handler.setFormatter(file_fmt)
logger.addHandler(file_handler)

# =========================
# Fonctions de journalisation
# =========================
def log_event(level: str, message: str):
    """
    Écrit dans logs.csv et via logger standard.
    level: 'INFO' | 'WARNING' | 'ERROR'
    """
    ts = utc_timestamp()
    # Logger console/fichier
    if level.upper() == 'INFO':
        logger.info(message)
    elif level.upper() == 'WARNING':
        logger.warning(message)
    elif level.upper() == 'ERROR':
        logger.error(message)
    else:
        logger.info(message)

    # Ajout dans logs.csv
    with open(LOG_CSV_PATH, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([ts, level.upper(), message])

def append_summary_md(text: str):
    """
    Append dans summary.md. Utiliser des chaînes brutes pour
    inclure LaTeX sans warnings d’échappement.
    """
    with open(SUMMARY_MD_PATH, 'a', encoding='utf-8') as f:
        f.write(text + '\n')

# =========================
# Banner de session
# =========================
session_header = r"""# Session Log T_log V0.1

- Session started: {ts}
- Conventions: bias=0 by default, seeds fixed (42), outputs in results/
""".format(ts=utc_timestamp())

# Écrire header si le fichier est nouveau
if not os.path.isfile(SUMMARY_MD_PATH) or os.path.getsize(SUMMARY_MD_PATH) == 0:
    append_summary_md(session_header)

# =========================
# Vérification environnement
# =========================
plt.figure(figsize=(4, 3))
x = np.linspace(0, 2*np.pi, 200)
plt.plot(x, np.sin(x), color='steelblue', lw=2)
plt.title('Env check plot')
plt.tight_layout()
env_plot_path = os.path.join('results', 'env_check_plot.png')
plt.savefig(env_plot_path, dpi=150)
plt.close()

# =========================
# Logs init
# =========================
log_event('INFO', 'Bloc 1 prêt: imports, seeds, dossiers et logger configurés.')
log_event('INFO', f'Plot de vérification sauvegardé: {env_plot_path}')

append_summary_md(r"""---
## Bloc 1 — Préparation
- Imports, seeds, dossiers et logger configurés.
- Env check plot: results/env_check_plot.png
""")

# Affichage de confirmation minimal
print("Bloc 1 OK — Dossiers et logger prêts.")
print(f"Seeds fixés: {SEED}")
print(f"Logs: {LOG_CSV_PATH}")
print(f"Summary: {SUMMARY_MD_PATH}")


2025-11-11 02:14:33,438 [INFO] Bloc 1 prêt: imports, seeds, dossiers et logger configurés.
2025-11-11 02:14:33,440 [INFO] Plot de vérification sauvegardé: results\env_check_plot.png


Bloc 1 OK — Dossiers et logger prêts.
Seeds fixés: 42
Logs: logs\logs.csv
Summary: logs\summary.md


Quick summary: We'll move on to Block 2 — Data Acquisition. The goal is to unzip your Urban Air Quality & Climate Dataset (1958-2025).zip ZIP file into the data/ folder, verify its contents (CSV files, etc.), and log the operation.

Block 2 — Data Acquisition (Air Quality)
Here is the corresponding Python cell:

In [3]:
import os
import pandas as pd
import json
import zipfile
from datetime import datetime

# --- 0. INSTALLATION DE KAGGLE ---
# Cette ligne assure que la librairie Kaggle est installée
!pip install kaggle --quiet

# --- Dépendance Kaggle ---
try:
    # Tenter d'importer la librairie Kaggle
    import kaggle.api as kaggle_api
except ImportError:
    print("Échec de l'importation de 'kaggle' même après installation. Veuillez vérifier votre environnement.")
    raise
# ------------------------

# --- 1. CONFIGURATION ET FONCTIONS DE LOGGING ---

# Identifiants du Dataset Kaggle
KAGGLE_DATASET_ID = "krishd123/urban-air-quality-and-climate-dataset-1958-2025"
TARGET_FILE_NAME = "urban_climate.csv"

# Chemins de travail
DATA_DIR = 'data'
LOGS_DIR = 'logs'
RESULTS_DIR = 'results'

# Fichier de données après téléchargement/extraction
LOCAL_COPY = os.path.join(DATA_DIR, TARGET_FILE_NAME)
# Fichiers de log et de résultats
RESULT_PREVIEW = os.path.join(RESULTS_DIR, 'urban_climate_preview.csv')
LOGS_CSV = os.path.join(LOGS_DIR, 'logs.csv')
SUMMARY_MD = os.path.join(LOGS_DIR, 'summary.md')

# Création des dossiers
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)


def append_log(level, message):
    """Ajoute une entrée au fichier de log CSV et Markdown."""
    ts = datetime.utcnow().isoformat() + 'Z'
    entry = pd.DataFrame([{'timestamp': ts, 'level': level, 'message': message}])
    
    # Écriture du log
    try:
        if os.path.exists(LOGS_CSV):
            df_logs = pd.read_csv(LOGS_CSV)
            df_logs = pd.concat([df_logs, entry], ignore_index=True)
        else:
            df_logs = entry
            
        df_logs.to_csv(LOGS_CSV, index=False)
        with open(SUMMARY_MD, 'a', encoding='utf-8') as f:
            f.write(f'\n- {ts} **{level}**: {message}\n')
    except Exception as e:
        print(f"[ALERTE] Échec de l'écriture du log: {e}")

# Alias pour utiliser 'log_event' si désiré, tout en utilisant la fonction `append_log`
log_event = append_log


def find_and_auth_kaggle():
    """Tente de trouver les clés d'API et authentifie l'API Kaggle."""
    log_event('INFO', 'Tentative d\'authentification Kaggle...')
    
    # 1. Vérifier les variables d'environnement (méthode Colab/Notebook)
    if os.getenv('KAGGLE_USERNAME') and os.getenv('KAGGLE_KEY'):
        log_event('INFO', 'Authentification via variables d\'environnement (KAGGLE_USERNAME/KEY).')
    
    # 2. Chercher le fichier kaggle.json
    else:
        locations = [
            os.path.join(os.path.expanduser('~'), '.kaggle', 'kaggle.json'), # Emplacement standard
            os.path.join(os.getcwd(), 'kaggle.json')                       # Répertoire actuel
        ]
        
        found = False
        for loc in locations:
            if os.path.exists(loc):
                try:
                    with open(loc, 'r') as f:
                        config = json.load(f)
                        username = config.get('username')
                        key = config.get('key')
                        if username and key:
                            os.environ['KAGGLE_USERNAME'] = username
                            os.environ['KAGGLE_KEY'] = key
                            log_event('INFO', f'Clés lues et définies via {loc}.')
                            found = True
                            break
                except (json.JSONDecodeError, Exception):
                    # Fichier trouvé mais invalide, on continue la recherche
                    continue
        
        if not found:
            log_event('ERROR', "Fichier kaggle.json introuvable. Veuillez le placer dans ~/.kaggle/ ou le répertoire courant.")
            return False

    # 3. Authentifier l'API
    try:
        kaggle_api.authenticate()
        log_event('SUCCESS', 'Authentification Kaggle réussie.')
        return True
    except Exception as e:
        log_event('ERROR', f'Échec de l\'authentification de l\'API: {e}')
        return False


# --- 2. AUTHENTIFICATION ET TÉLÉCHARGEMENT ---
try:
    if not find_and_auth_kaggle():
        # Lever une exception si l'authentification échoue
        raise RuntimeError("Processus annulé. Échec de la configuration Kaggle. Assurez-vous d'avoir configuré votre API Key.")
    
    print(f"\nDébut du téléchargement de : {KAGGLE_DATASET_ID}")
    log_event('INFO', f"Téléchargement et décompression du dataset : {KAGGLE_DATASET_ID}")
    
    # Télécharger et décompresser directement le dataset dans le dossier 'data/'
    kaggle_api.dataset_download_files(
        KAGGLE_DATASET_ID, 
        path=DATA_DIR, 
        unzip=True,
        # 'force=True' pour re-télécharger si le fichier existe déjà (reproductibilité)
        force=True, 
        quiet=True # Rendre l'API Kaggle moins verbeuse
    )
    
    if not os.path.exists(LOCAL_COPY):
        raise FileNotFoundError(f"Le fichier {TARGET_FILE_NAME} est introuvable après l'extraction du dataset Kaggle. Le dataset pourrait avoir une structure différente.")
        
    log_event('SUCCESS', f"Téléchargement et préparation du fichier : {LOCAL_COPY}")
    print(f"Téléchargement terminé. Fichier cible : {LOCAL_COPY}")


    # --- 3. LECTURE ROBUSTE ET ANALYSE DU FICHIER ---
    
    read_errors = []
    df = None
    log_event('INFO', f"Tentative de lecture du CSV : {LOCAL_COPY}")
    
    # Tentative 1: Standard (utf-8, comma)
    try:
        df = pd.read_csv(LOCAL_COPY)
    except Exception as e1:
        read_errors.append(f"Standard: {e1}")
        
        # Tentative 2: utf-8, semicolon
        try:
            df = pd.read_csv(LOCAL_COPY, encoding='utf-8', sep=';')
        except Exception as e2:
            read_errors.append(f"UTF-8/Semicolon: {e2}")
            
            # Tentative 3: latin1, standard sep
            try:
                df = pd.read_csv(LOCAL_COPY, encoding='latin1')
            except Exception as e3:
                read_errors.append(f"Latin1: {e3}")
                
                # Échec total de lecture
                raise RuntimeError(f"Impossible de lire le CSV ({LOCAL_COPY}). Échecs: {'; '.join(read_errors)}")
    
    # Si la lecture est réussie:
    n_rows = df.shape[0]
    n_cols = df.shape[1]
    cols = list(df.columns)
    missing_counts = df.isna().sum()

    # Sauvegarder un aperçu
    df.head(200).to_csv(RESULT_PREVIEW, index=False)

    # Log et affichage du succès
    log_event('SUCCESS', f'Chargement réussi: {LOCAL_COPY}; rows={n_rows}; cols={n_cols}')
    log_event('INFO', f'Colonnes détectées: {cols}')
    log_event('INFO', f'Missing per column (seulement > 0): {dict(missing_counts[missing_counts > 0])}') 
    
    print("\n" + "="*50)
    print(f"ANALYSE DU FICHIER {TARGET_FILE_NAME}")
    print(f" - Nombre d'enregistrements (n) : {n_rows}")
    print(f" - Nombre de colonnes : {n_cols}")
    print(" - Colonnes :", cols)
    print(f" - Aperçu sauvegardé : {RESULT_PREVIEW}")
    print("\nComptes de valeurs manquantes (seulement les colonnes non nulles) :")
    print(missing_counts[missing_counts > 0])
    print("="*50)
    
except Exception as e:
    log_event('CRITICAL', f'Échec du bloc de code: {e}')
    print("\n" + "#"*50)
    print("ÉCHEC CRITIQUE: Le code n'a pas pu s'exécuter.")
    print(f"Erreur: {e}")
    print(f"Vérifiez que vous avez configuré votre clé d'API Kaggle (fichier kaggle.json).")
    print("#"*50)
    raise

C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.p


Début du téléchargement de : krishd123/urban-air-quality-and-climate-dataset-1958-2025
Dataset URL: https://www.kaggle.com/datasets/krishd123/urban-air-quality-and-climate-dataset-1958-2025
Téléchargement terminé. Fichier cible : data\urban_climate.csv

ANALYSE DU FICHIER urban_climate.csv
 - Nombre d'enregistrements (n) : 11040
 - Nombre de colonnes : 12
 - Colonnes : ['city', 'country', 'latitude', 'longitude', 'year', 'month', 'temperature_celsius', 'humidity_percent', 'precipitation_mm', 'wind_speed_ms', 'urban_heat_island_intensity', 'data_source']
 - Aperçu sauvegardé : results\urban_climate_preview.csv

Comptes de valeurs manquantes (seulement les colonnes non nulles) :
Series([], dtype: int64)


C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.p

Perfect 👌, your **Block 2 — Data Acquisition** is validated: the ZIP file has been successfully unzipped and we now have 5 usable components:

- `metadata.json` → complete documentation
- `co2_emissions.csv` → direct CO₂ measurements (Mauna Loa, 1958–2025)
- `air_quality_global.csv` → PM2.5 and NO₂ for 20 cities (1999–2025)
- `urban_climate.csv` → urban climate variables (1980–2025)
- `ice_core_co2.csv` → paleoclimate reconstructions (2000 years BP)

---

### Next strategic choice
To apply your **T_log model V0.1**, we need to define:
- **n** = system size (number of observations or nodes)
- **d** = effective dimension (spectral, spatial, or chosen proxy)

👉 Two possible approaches:
1. **Air Quality (PM2.5, NO₂)**:
- n = number of measurement points (per city or global)
- d = effective temporal/spatial dimension (e.g., d=1 for time series, d≈2–3 if combining several cities as a graph)

2. **Direct CO₂ (Mauna Loa)**:
- n = number of months measured (≈800+)
- d = 1 (one-dimensional time series)

3. **Ice Core CO₂**:
- n = ~2000 years of data
- d = 1 (long time series)

---

**Quick Summary:** We'll tackle **Block 3 — Calculating T_log** on the `air_quality_global.csv` file. We'll load the PM2.5 data, choose a city (or the global set), define n as the number of valid observations, set d=1 (one-dimensional time series), then calculate and classify T_{log}.

---

### Block 3 — Calculating T_log (Air Quality Global, PM2.5)

Before starting the calculation of
𝑇
log
⁡
, it's more rigorous to check the state of the air_quality_global.csv file: structure, columns, missing values, duplicates, etc. This will constitute our Block 3a — Data Inspection and Validation.

Block 3a — Inspection of the air_quality_global.csv File

In [4]:
# Bloc 3a — Inspection et validation du fichier air_quality_global.csv

import pandas as pd

aq_path = "data/air_quality_global.csv"

# Charger un échantillon pour inspection
df_aq = pd.read_csv(aq_path)

# Aperçu général
print("=== Aperçu du dataset ===")
print(df_aq.head(10))   # premières lignes
print("\nColonnes disponibles :", df_aq.columns.tolist())
print("Nombre total de lignes :", len(df_aq))

# Vérification des types et valeurs manquantes
print("\n=== Info ===")
print(df_aq.info())

print("\n=== Valeurs manquantes par colonne ===")
print(df_aq.isna().sum())

# Vérification des doublons
nb_duplicates = df_aq.duplicated().sum()
print(f"\nNombre de doublons détectés : {nb_duplicates}")

# Aperçu statistique des colonnes numériques
print("\n=== Statistiques descriptives ===")
print(df_aq.describe(include='all').transpose().head(20))

# Log
log_event("INFO", f"Inspection du fichier {aq_path} effectuée : {len(df_aq)} lignes, {nb_duplicates} doublons")
append_summary_md(f"- {utc_timestamp()} [INFO] Inspection du fichier {aq_path} : {len(df_aq)} lignes, {nb_duplicates} doublons")


C:\Users\zackd\AppData\Local\Temp\ipykernel_7700\2067266531.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


=== Aperçu du dataset ===
       city country  latitude  longitude  year  month  pm25_ugm3  no2_ugm3  \
0  New York     USA   40.7128    -74.006  1999      1      18.11     35.98   
1  New York     USA   40.7128    -74.006  1999      2      27.79     17.71   
2  New York     USA   40.7128    -74.006  1999      3      12.05     40.99   
3  New York     USA   40.7128    -74.006  1999      4      35.25     17.18   
4  New York     USA   40.7128    -74.006  1999      5      38.39     25.07   
5  New York     USA   40.7128    -74.006  1999      6      14.89     28.95   
6  New York     USA   40.7128    -74.006  1999      7      19.66     27.85   
7  New York     USA   40.7128    -74.006  1999      8      10.00     26.14   
8  New York     USA   40.7128    -74.006  1999      9      15.04     38.56   
9  New York     USA   40.7128    -74.006  1999     10      15.32     29.50   

  data_quality           measurement_method data_source  
0     Moderate  Reference/Equivalent Method     EPA_AQS  

Quick summary: We will calculate
𝑇
log
⁡
in global mode on the entire PM2.5 dataset (n = 6480, d = 1), then prepare a variant for each city (e.g., New York, n = 324).

Block 3b — Calculation of T_log (global PM2.5)